In [21]:
import os
import folium
from folium import FeatureGroup,LayerControl, Map, Marker
import urllib
from urllib.request import urlopen
import json
import geopandas as gp
from pathlib import Path
import pandas as pd
import fiona
import branca

ImportError: cannot import name 'LayerGroup' from 'folium' (C:\Users\mferb\Anaconda3\lib\site-packages\folium\__init__.py)

In [3]:
with open('../data/cleandata/harris_co_clean.geojson') as response:
    tracts_data = json.load(response)

In [69]:
##ALL DATA
join_df = pd.read_csv('../data/cleandata/join_data.csv',dtype={"FIPS": str})
join_df.head()

# ##SVI DATA TO DataFrame and CLEANED
# svi_df = join_df[['FIPS','SVI INDEX']]
# svi_df = svi_df.rename(columns={'FIPS':'FIPS','SVI INDEX':'SVI'})
# svi_df = svi_df[(svi_df['SVI'] >= 0) & (svi_df['SVI'] <= 1)]

# ##school DATA TO CHOROPLETH IMPORTED AND CLEANED
# school_df = join_df[['FIPS','Rating']]

# # FLOOD DATA
# flood_df = join_df[['FIPS','Pct_in_100yr']]

# ## EVICTION DATA
# eviction_df = join_df[['FIPS','eviction-rate']]

# ## FOOD DESERT
# food_desert_df = join_df[['FIPS','food_desert']]

## POVERTY RATE
#poverty_df = join_df[['FIPS','poverty-rate']]


## MEDIAN PROPERTY VALUE
property_value_df = join_df[['FIPS','median-property-value']]

## ROAD CLOSURES
road_df = pd.read_csv('../data/cleandata/road_flooding_imelda.csv',dtype={"FIPS": str})
road_df.head()

,Intersection,Lat,Long,FIPS
0,I-10 and McCarty,29.778123,-95.284054,48201212400
1,I-10 and San Jacinto,29.767983,-95.356149,48201210100
2,I-10 and Waco St,29.772891,-95.326537,48201211300
3,I-10 and Gessner Rd,29.784885,-95.543884,48201522302
4,I-10 and 610W,29.798637,-95.451017,48201520500


In [74]:
#folium.Marker([29.809246, -95.281791], popup='7969 Fields St').add_to(feature_group2)
for index, item in road_df.iterrows():
    lat = item.get('Lat')
    long = item.get('Long')
    name = item.get('Intersection')
    geo = f'{lat},{long}'

    #folium.Marker([lat,long], popup=name).add_to(map)


I-10 and McCarty
I-10 and San Jacinto
I-10 and Waco St
I-10 and Gessner Rd
I-10 and 610W
I-10 and Westcott
I-10 Echo ln
I-10 and Washington
I-45 and Quitman
I-45 and Airline
I-45 and North St
I-45 and Parker
I-45 and Richey
I-45 and Mount Houston Rd
I-10 and Milam
I-45 and Main St
I-610 and Wayside
I-610 and Hirsch
I-610 and Scott St
I-610 and Old Katy
I-69 and Beltway 8 N
I-69 and I-610
I-69 and Jenson
I-69 and Will Clayton Pkwy
I-69 and Greens Rd
Beltway 8 and Tidwell
Beltway 8 and Ralston
Beltway 8 and Hammerly


In [26]:
# from branca.element import MacroElement

# from jinja2 import Template
# class BindColormap(MacroElement):
#     """Binds a colormap to a given layer.

#     Parameters
#     ----------
#     colormap : branca.colormap.ColorMap
#         The colormap to bind.
#     """
#     def __init__(self, layer, colormap):
#         super(BindColormap, self).__init__()
#         self.layer = layer
#         self.colormap = colormap
#         self._template = Template(u"""
#         {% macro script(this, kwargs) %}
#             {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
#             {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
#                 if (eventLayer.layer == {{this.layer.get_name()}}) {
#                     {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
#                 }});
#             {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
#                 if (eventLayer.layer == {{this.layer.get_name()}}) {
#                     {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
#                 }});
#         {% endmacro %}
#         """)  # noqa
# colormap_svi = branca.colormap.LinearColormap(['y','orange','r'], vmin=0, vmax=1, caption='SVI')

In [81]:


map = folium.Map([29.7604, -95.358421], tiles = "OpenStreetMap", zoom_start=10)



folium.Choropleth(
    
    geo_data=tracts_data,
    name='SVI',
    data=svi_df,
    columns=['FIPS', 'SVI'],
    key_on='feature.properties.FIPS',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',
    nan_fill_opacity=0.3,
    legend_name='SVI INDEX',
    colormap = colormap_svi
).add_to(map)


# folium.Choropleth(
#    geo_data=tracts_data,
#    name='School Rankings',
#    data=school_df,
#    columns=['FIPS', 'Rating'],
#    key_on='feature.properties.FIPS',
#    fill_color='YlGn',
#    fill_opacity=0.7,
#    line_opacity=0.2,
#    nan_fill_color='white',
#    nan_fill_opacity=0.3,
#    legend_name='School Ratings',
#    show=False,
# ).add_to(map)

folium.Choropleth(
   geo_data=tracts_data,
   name='Flood Zones',
   data=flood_df,
   columns=['FIPS', 'Pct_in_100yr'],
   key_on='feature.properties.FIPS',
   fill_color='YlGn',
   fill_opacity=0.7,
   line_opacity=0.2,
   nan_fill_color='white',
   nan_fill_opacity=0.3,
   legend_name='Pct. of Houses in 100yr Flood Zone',
).add_to(map)

# folium.Choropleth(
#    geo_data=tracts_data,
#    name='Eviction Rate',
#    data=eviction_df,
#    columns=['FIPS', 'eviction-rate'],
#    key_on='feature.properties.FIPS',
#    fill_color='YlGn',
#    fill_opacity=0.7,
#    line_opacity=0.2,
#    nan_fill_color='white',
#    nan_fill_opacity=0.3,
#    legend_name='Eviction Rate',
# ).add_to(map)

# folium.Choropleth(
#    geo_data=tracts_data,
#    name='Food Desert',
#    data=food_desert_df,
#    columns=['FIPS', 'food_desert'],
#    key_on='feature.properties.FIPS',
#    fill_color='YlGn',
#    fill_opacity=0.7,
#    line_opacity=0.2,
#    nan_fill_color='white',
#    nan_fill_opacity=0.3,
#    legend_name='Food Desert',
# ).add_to(map)

# folium.Choropleth(
#    geo_data=tracts_data,
#    name='Poverty Rate',
#    data=poverty_df,
#    columns=['FIPS', 'poverty-rate'],
#    key_on='feature.properties.FIPS',
#    fill_color='YlGn',
#    fill_opacity=0.7,
#    line_opacity=0.2,
#    nan_fill_color='white',
#    nan_fill_opacity=0.3,
#    legend_name='Poverty Rate',
# ).add_to(map)


# folium.Choropleth(
#    geo_data=tracts_data,
#    name='Median Property Value',
#    data=property_value_df,
#    columns=['FIPS', 'median-property-value'],
#    key_on='feature.properties.FIPS',
#    fill_color='YlGn',
#    fill_opacity=0.7,
#    line_opacity=0.2,
#    nan_fill_color='white',
#    nan_fill_opacity=0.3,
#    legend_name='Median Property Value',
# ).add_to(map)

##HOUSE PIN LAT LONG
house_pin = FeatureGroup(name='House Pin')
folium.Marker([29.809246, -95.281791], popup='7969 Fields St').add_to(house_pin)
house_pin.add_to(map)


##ROAD CLOSURE PINS
road_pin = FeatureGroup(name = 'Road Closures Imelda')
for index, item in road_df.iterrows():
    lat = item.get('Lat')
    long = item.get('Long')
    name = item.get('Intersection')
    geo = f'{lat},{long}'

    folium.Marker([lat,long], popup=name,icon = folium.Icon(color='red')).add_to(road_pin)

road_pin.add_to(map)

LayerControl(collapsed=False).add_to(map)

map.save(os.path.join('../data/rawdata/CHOROPLETH_SVI.html'))

map